In [ ]:
from Raspi_MotorHAT import Raspi_MotorHAT
import keyboard
import threading
import time
import cv2
import RPi.GPIO as GPIO
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
mh = Raspi_MotorHAT(addr=0x6f)


def motor_go():
    motor1 = mh.getMotor(1)
    motor1.setSpeed(50)
    motor1.run(Raspi_MotorHAT.FORWARD)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(50)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(50)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(50)
    motor4.run(Raspi_MotorHAT.FORWARD)

def motor_stop():
    motor1 = mh.getMotor(1)
    motor1.setSpeed(50)
    motor1.run(Raspi_MotorHAT.RELEASE)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(50)
    motor2.run(Raspi_MotorHAT.RELEASE)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(50)
    motor3.run(Raspi_MotorHAT.RELEASE)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(50)
    motor4.run(Raspi_MotorHAT.RELEASE)
    
def motor_right():
    motor1 = mh.getMotor(1)
    motor1.setSpeed(70)
    motor1.run(Raspi_MotorHAT.FORWARD)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(15)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(70)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(15)
    motor4.run(Raspi_MotorHAT.FORWARD)  
    
def motor_left():
    motor1 = mh.getMotor(1)
    motor1.setSpeed(15)
    motor1.run(Raspi_MotorHAT.FORWARD)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(70)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(15)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(70)
    motor4.run(Raspi_MotorHAT.FORWARD)

# Pretrained classes in the model
classNames = {0: 'background',
              1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus',
              7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant',
              13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat',
              18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear',
              24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag',
              32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
              37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
              41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
              46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
              51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
              56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
              61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',
              67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
              75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
              80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
              86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush'}


def id_class_name(class_id, classes):
    for key, value in classes.items():
        if class_id == key:
            return value
        
def img_preprocess(image):
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    image = cv2.resize(image, (200,66))
    image = cv2.GaussianBlur(image,(5,5),0)
    _,image = cv2.threshold(image,160,255,cv2.THRESH_BINARY_INV)
    image = image / 255
    return image

camera = cv2.VideoCapture(0)
camera.set(3, 640)
camera.set(4, 480)

_, image = camera.read()
image_ok = 0

box_size = 0
carState = "stop"

def opencvdnn_thread():
    global image
    global image_ok
    global box_size
    global carState
    model = cv2.dnn.readNetFromTensorflow('/home/jh/model/frozen_inference_graph.pb',
                                      '/home/jh/model/ssd_mobilenet_v2_coco_2018_03_29.pbtxt')
 
    while True:
        if image_ok == 1:
            imagednn = image
            image_height, image_width, _ = imagednn.shape
            
            model.setInput(cv2.dnn.blobFromImage(imagednn, size=(250, 250), swapRB=True))
            output = model.forward()
            # print(output[0,0,:,:].shape)
            for detection in output[0, 0, :, :]:
                confidence = detection[2]
                if confidence > .5:
                    class_id = detection[1]
                    class_name=id_class_name(class_id,classNames)
                    if class_name == "person":
                        print(str(str(class_id) + " " + str(detection[2])  + " " + class_name))
                        box_x = detection[3] * image_width
                        box_y = detection[4] * image_height
                        box_width = detection[5] * image_width
                        box_height = detection[6] * image_height
                        box_size = box_width * box_height
                        #print("box_size:",box_size)
                        
                        carState = "stop"
                        print("auto stop")
                
                        cv2.rectangle(imagednn, (int(box_x), int(box_y)), (int(box_width), int(box_height)), (23, 230, 210), thickness=1)
                        cv2.putText(imagednn,class_name ,(int(box_x), int(box_y+.05*image_height)),cv2.FONT_HERSHEY_SIMPLEX,(.005*image_width),(0, 0, 255))
            
            #cv2.imshow('imagednn', imagednn)
            image_ok = 0
        

def main():
    global image
    global image_ok
    global carState
    
    model_path = '/home/jh/model/1205-2.h5'
    model = load_model(model_path)
    
    try:
        while True:
            keyValue = cv2.waitKey(1)
        
            if keyboard.is_pressed("p"):
                break
            elif keyboard.is_pressed("w"):
                print("go")
                carState = "go"
            elif keyboard.is_pressed("s"):
                print("stop")
                carState = "stop"

            
            image_ok = 0
            _, image = camera.read()
            image_ok = 1
            
            preprocessed = img_preprocess(image)
            
            #cv2.imshow('pre', preprocessed)
            
            X = np.asarray([preprocessed])
            steering_angle = int(model.predict(X)[0])
            print("predict angle:",steering_angle)
                
            if carState == "go":
                if steering_angle >= 80 and steering_angle <= 105:
                    print("go")
                    motor_go()
                elif steering_angle > 106:
                    print("right")
                    motor_right()
                elif steering_angle < 79:
                    print("left")
                    motor_left()
            elif carState == "stop":
                motor_stop()
                time.sleep(3)
                carState == "go"
            
    except KeyboardInterrupt:
        pass

if __name__ == '__main__':
    task1 = threading.Thread(target = opencvdnn_thread)
    task1.start()
    main()
    cv2.destroyAllWindows()
